In [176]:
import PyPDF2
import urllib
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import io

In [204]:
def gen_valid_dates():
    d_to_get = []
    for yr in range(2015, 2021):
        syr = str(yr)
        for month in range(1, 13):
            smth = str(month).zfill(2)
            d_to_get.append(syr + '_' + smth)
    return d_to_get[0:9] + d_to_get[10:62] + [d_to_get[63]]

def process_soup(soup, date):
    trs = soup.find_all('table')[1].find_all('tr')
    results = []
    for i in range(1, len(trs)):
        cur_tr = trs[i]    
        tds = cur_tr.find_all("td")
        sub_dict = {}
        sub_dict['link'] = tds[0].find('a')['href']
        sub_dict['appellant'] = tds[1].text
        sub_dict['county'] = tds[2].text
        sub_dict['reason_code'] = tds[3].text
        docket = tds[0].text.split('-')
        sub_dict['prop_type'] = docket[2]
        sub_dict['valuation_class'] = docket[3]
        sub_dict['docket_name'] = docket[0] + '-' + docket[1]
        sub_dict['date'] = date
        results.append(sub_dict)
    return pd.DataFrame(results)

def get_itemized_decisions():
    valid_dates = gen_valid_dates()
    for i, date in enumerate(valid_dates):
        print(date)
        response = requests.get('http://www.ptab.illinois.gov/decisions/default.asp?sdf=' + date)
        time.sleep(5)
        soup = BeautifulSoup(response.content, 'html.parser')
        if i == 0:
            result = process_soup(soup, date)
        else:
            result = result.append(process_soup(soup, date))
    return result

In [205]:
decisions = get_itemized_decisions()
decisions.to_csv('decisions.csv', index=False)

2015_01
2015_02
2015_03
2015_04
2015_05
2015_06
2015_07
2015_08
2015_09
2015_11
2015_12
2016_01
2016_02
2016_03
2016_04
2016_05
2016_06
2016_07
2016_08
2016_09
2016_10
2016_11
2016_12
2017_01
2017_02
2017_03
2017_04
2017_05
2017_06
2017_07
2017_08
2017_09
2017_10
2017_11
2017_12
2018_01
2018_02
2018_03
2018_04
2018_05
2018_06
2018_07
2018_08
2018_09
2018_10
2018_11
2018_12
2019_01
2019_02
2019_03
2019_04
2019_05
2019_06
2019_07
2019_08
2019_09
2019_10
2019_11
2019_12
2020_01
2020_02
2020_04


In [206]:
decisions.iloc[0]

link               http://www.ptab.illinois.gov/web/Decisions/201...
appellant                                                 Busey Bank
county                                                     Champaign
reason_code                                                        2
prop_type                                                          C
valuation_class                                                    2
docket_name                                               2012-00035
date                                                         2015_01
Name: 0, dtype: object

In [226]:
def link_to_text(link):
    response = requests.get(link) 
    with io.BytesIO(response.content) as f:
        pdf = PyPDF2.PdfFileReader(f)
        num_page = pdf.getNumPages()
        full_str = ''
        for i in range(num_page):
            page = pdf.getPage(i)
            text = page.extractText()
            full_str = full_str + text
    return full_str

def process_entry(entry):
    try:
        text = link_to_text(entry['link'])
        f = open('files/' + entry['docket_name'] + '.txt', "w+")
        f.write(text)
        f.close()
        entry.to_frame().T.to_csv('success.csv', mode='a', header=False, index=False)
    except:
        entry.to_frame().T.to_csv('failed.csv', mode='a', header=False, index=False)

def process_decisions(decisions):
    for index, row in decisions.iterrows():
        if index % 100 == 0:
            print(index)
        process_entry(row)
        time.sleep(0.5)

In [ ]:
process_decisions(decisions)

0
100
200
300


Reason Closed Key:	1‑Decision/Hearing | 2‑Decision/Evidence | 3‑Stipulation | 4‑Dismissal | 5‑Withdrawal | 6‑Duplicate | 7‑Premature | 8‑Rollover | 9‑Incomplete | A‑Void | B‑Dismissal/Lack Standing | C‑Dismissal/FTA